In [29]:
# P_from_node_i_to_node_j
# Pij = (1/bij)*(x[i]-x[j])
# H[line_id,i] = 1/bij
# H[line_id,j] = -1/bij
A_real = np.real(net._ppc['internal']['Bbus'].A)
H = np.zeros((net.line.shape[0]+net.trafo.shape[0], net.bus.shape[0]))
for i in range(0, net.line.shape[0]):
    H[i, net.line.from_bus.values[i]] = 1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]
    H[i, net.line.to_bus.values[i]] = -1/A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]

for i in range(net.line.shape[0]+1, net.line.shape[0]+net.trafo.shape[0]):
    H[i, net.trafo.hv_bus.values[i-net.line.shape[0]]] = 1/A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]
    H[i, net.trafo.lv_bus.values[i-net.line.shape[0]]] = -1/A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]    

In [132]:
# method: state estimation
# estimate x given z, where x is not (#nodes,1), but (#nodes-1,1)
# x = [theta_0, theta_1, theta_2, ...,]
# theta_0 = -10
# theta_1 = 0 deg
# input params: H, z, cov

H_est = np.copy(H[:,1:])
# z_t_mat[:,0] is z from params
x_est = np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(H_est),H_est)),np.transpose(H_est)),z_t_mat[:,0])
x_est = np.hstack((0, x_est))

# r = z - H*x for a single point in time
residuals = z_t_mat[:,0] - np.matmul(H, x_est)
normalized_residuals = residuals/np.sqrt(cov[0,0])

# return x_est, normalized_residuals for a single point in time

In [152]:
# least effort based on norm-1
# min norm-1(a) with respect to a and c, where a is in z+a and c is in x+c
# subject to:
# a = H*c
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1

# input params: H, z, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

m = Model('least effort with norm 1')

# define decision variables: a and c vectors

a  = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c  = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]
a_positive = [m.addVar(lb=0.0,name='a_pos%d' % i) for i in range(H.shape[0])]
a_negative = [m.addVar(ub=0.0,name='a_neg%d' % i) for i in range(H.shape[0])]

m.update()

# constraint: a[i] = a_positive[i] - a_negative[i]
# not a constraint: |a[i]| = a_positive[i] + a_negative[i]
for i in range(0,H.shape[0]):
    m.addConstr(a[i] == a_positive[i] + a_negative[i])

# 1st constraint: a[k] = delta_a
m.addConstr(a[k] == delta_a)

# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: a = H*c
for i in range(0,H.shape[0]):
    m.addConstr(a[i] == np.matmul(H,c)[i])

# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(z[0]-z[1] for z in zip(a_positive, a_negative)), GRB.MINIMIZE)

m.update()

m.optimize()

#least_effort_a_norm_1 = m.getVars()[0:H.shape[0]]
least_effort_a_norm_1 = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
least_effort_a_norm_1 = np.asarray(least_effort_a_norm_1)

# where z_t_mat[:,0] see z
z_perturbed = z_t_mat[:,0] + least_effort_a_norm_1

# returns least_effort_a_norm_1, z_perturbed for a single point in time

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 109 rows, 74 columns and 187 nonzeros
Model fingerprint: 0xcbce9674
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+02]
Presolve removed 100 rows and 57 columns
Presolve time: 0.01s
Presolved: 9 rows, 17 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e-01   4.105379e-02   0.000000e+00      0s
       6    4.2803658e-01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.03 seconds (0.00 work units)
Optimal objective  4.280365769e-01


In [153]:
least_effort_a_norm_1

array([0.        , 0.12827237, 0.        , 0.10140327, 0.1       ,
       0.09836094, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [133]:
# z_t_mat (#meas, T)
# z_perturbed_t_mat = np.zeros((H.shape[0],-1)) 
# for i range(0,T):
# if i >= t1 and i <= t2
# call method that computes a which returns z_perturbed
# z_perturbed_t_mat = np.hstack((z_perturbed_t_mat, z_perturbed))
# outside the for loop
# z_perturbed_t_mat = np.delete(z_perturbed_t_mat, 0, 1)

In [160]:
# least effort based on big-M, where M is a hyper-parameter
# min sum(y) with respect to a, c, and y, where a is in z+a and c is in x+c
# subject to:
# a = H*c
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1
# y[i] where i going from 0 to #meas-1 and y[i] is integer
# -y[i]*M <= a[i] <= y[i]*M
# if y[i] = 0, then 0 <= a[i] <= 0
# if y[i] = 1, then -M <= a[i] <= M

# input params: H, z, M, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

M = 10**4

m = Model('least effort with big-M')

# define decision variables: a and c vectors

a = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]
y = [m.addVar(vtype=gurobipy.GRB.BINARY, name='y%d' % i) for i in range(H.shape[0])]

m.update()

# -y[i]*M <= a[i] <= y[i]*M
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= y[i]*M)
    m.addConstr(a[i] >= -y[i]*M)

# 1st constraint: a[k] = delta_a
m.addConstr(a[k] == delta_a)

# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: a = H*c
for i in range(0,H.shape[0]):
    m.addConstr(a[i] == np.matmul(H,c)[i])

# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(y), GRB.MINIMIZE)

m.update()

m.optimize()

least_effort_a_big_M = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
least_effort_a_big_M = np.asarray(least_effort_a_big_M)

# where z_t_mat[:,0] see z
z_perturbed = z_t_mat[:,0] + least_effort_a_big_M

# returns least_effort_a_norm_1, z_perturbed for a single point in time

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 129 rows, 54 columns and 207 nonzeros
Model fingerprint: 0x8e803430
Variable types: 34 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+02]
Found heuristic solution: objective 5.0000000
Presolve removed 107 rows and 23 columns
Presolve time: 0.00s
Presolved: 22 rows, 31 columns, 60 nonzeros
Variable types: 17 continuous, 14 integer (14 binary)

Root relaxation: objective 3.005751e+00, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.00575    0    1    5.00000    3.00575  39.9%     -    0s
H    0     0                       4.0000000

In [161]:
least_effort_a_big_M

array([0.        , 0.12827237, 0.        , 0.        , 0.1       ,
       0.        , 0.02421785, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.30994848, 0.        , 0.        , 0.        ])

In [162]:
least_effort_a_norm_1

array([0.        , 0.12827237, 0.        , 0.10140327, 0.1       ,
       0.09836094, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])